In [1]:
import spacy
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import re

In [2]:
#pip install tensorflow

In [3]:
nlp = spacy.load("en_core_web_md")

In [4]:
df = pd.read_csv('trump_insult_tweets_2014_to_2021.csv',index_col='Unnamed: 0')
df.describe()

,date,target,insult,tweet
count,10360,10358,10360,10360
unique,1573,866,6729,5673
top,2020-10-12,the-media,Fake News,"The Fake News Networks, those that knowingly h..."
freq,45,1287,431,16


In [5]:
df['clean_tweets'] = df['tweet'].apply(lambda x: re.sub(r'http\S+', '', str(x)))

In [6]:
clean_tweets = np.array(df['clean_tweets'][df['clean_tweets'] != ''].unique())

In [7]:
np.array([len(tweet) for tweet in clean_tweets]).argmin()

5254

In [8]:
np.sort(np.array([len(tweet) for tweet in df['clean_tweets'].unique()]))

array([  0,   8,  10, ..., 281, 283, 284])

In [9]:
docs = [nlp(x) for x in clean_tweets]
print(docs[:10])

[Can you believe this fool, Dr. Thomas Frieden of CDC, just stated, "anyone with fever should be asked if they have been in West Africa" DOPE, Big time in U.S. today - MAKE AMERICA GREAT AGAIN! Politicians are all talk and no action - they can never bring us back., Politician @SenatorCardin didn't like that I said Baltimore needs jobs & spirit. It's politicians like Cardin that have destroyed Baltimore., For the nonbeliever, here is a photo of @Neilyoung in my office and his $$ request—total hypocrite. , .@Neilyoung’s song, “Rockin’ In The Free World” was just one of 10 songs used as background music. Didn’t love it anyway., Uncomfortable looking NBC reporter Willie Geist calls me to ask for favors and then mockingly smiles when he is told of my high poll numbers, Just out, the new nationwide @FoxNews poll has me alone in 2nd place, closely behind Jeb Bush-but Bush will NEVER Make America Great Again!, The ratings for The View are really low. Nicole Wallace and Molly Sims are a disaste

In [10]:
#[token for token in docs[3]

In [11]:
len(docs[3])

22

In [12]:
seq_len = 2

In [13]:
X_data = []
y_data = []

In [14]:
for doc in docs:
    for j in range(len(doc) - seq_len):
        
        in_seq = doc[j : j + seq_len]
        out_seq = doc[j + seq_len]
        X_data.append([doc.vocab.strings[token.text] for token in in_seq])        
        y_data.append(doc.vocab.strings[out_seq.text])
        

In [15]:
doc.vocab.strings[str(y_data[100])]

2457247847353624091

In [16]:
n_patterns = len(X_data)

In [17]:
X = np.reshape(X_data, (n_patterns, seq_len, 1))
y = np.asarray(pd.get_dummies(y_data))

In [18]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]


In [20]:
model.fit(X, y, epochs=30, batch_size=256, callbacks=desired_callbacks)


Epoch 1/30
811/811 [==============================] - 112s 135ms/step - loss: 7.1253

Epoch 00001: loss improved from inf to 6.87142, saving model to model_weights_saved.hdf5
Epoch 2/30
811/811 [==============================] - 109s 135ms/step - loss: 6.7249

Epoch 00002: loss improved from 6.87142 to 6.73362, saving model to model_weights_saved.hdf5
Epoch 3/30
811/811 [==============================] - 105s 129ms/step - loss: 6.7144

Epoch 00003: loss improved from 6.73362 to 6.72474, saving model to model_weights_saved.hdf5
Epoch 4/30
811/811 [==============================] - 87s 107ms/step - loss: 6.7192

Epoch 00004: loss improved from 6.72474 to 6.72189, saving model to model_weights_saved.hdf5
Epoch 5/30
811/811 [==============================] - 86s 107ms/step - loss: 6.7070

Epoch 00005: loss improved from 6.72189 to 6.72009, saving model to model_weights_saved.hdf5
Epoch 6/30
811/811 [==============================] - 86s 107ms/step - loss: 6.7051

Epoch 00006: loss improved

In [21]:
pattern = [doc.vocab.strings['I'], doc.vocab.strings['want']]

In [22]:
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
#     x = x / float(vocab_len)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    print(index)
    result = doc.vocab.strings[index]
    seq_in = [doc.vocab.strings[value] for value in pattern]
    # CONVERT INDEX TO COL NAME TO RESTORE SPACY HASH 

    pattern.append(index)
    pattern = pattern[1:len(pattern)]


1757


KeyError: "[E018] Can't retrieve string for hash '1757'. This usually refers to an issue with the `Vocab` or `StringStore`."

In [ ]:
print(y)